In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/dataset1/armut_data.csv


## Türkiye’nin en büyük online hizmet platformu olan Armut, hizmet verenler ile hizmet almak isteyenleri buluşturmaktadır.
## Bilgisayarın veya akıllı telefonunun üzerinden birkaç dokunuşla temizlik, tadilat, nakliyat gibi hizmetlere kolayca ulaşılmasını sağlamaktadır.Hizmet alan kullanıcıları ve bu kullanıcıların almış oldukları servis ve kategorileri içeren veri setini kullanarak AssociationRule Learning ile ürün tavsiye sistemi oluşturulmak istenmektedir

# **1. Data Preparation**

# Veri seti müşterilerin aldıkları servislerden ve bu servislerin kategorilerinden oluşmaktadır.
# Alınan her hizmetin tarih ve saat bilgisini içermektedir.

## UserId: Müşteri numarası
## ServiceId: Her kategoriye ait anonimleştirilmiş servislerdir. (Örnek : Temizlik kategorisi altında koltuk yıkama servisi)
## Bir ServiceId farklı kategoriler altında bulanabilir ve farklı kategoriler altında farklı servisleri ifade eder.
### (Örnek: CategoryId’si 7 ServiceId’si 4 olan hizmet petek temizliği iken CategoryId’si 2 ServiceId’si 4 olan hizmet mobilya montaj)
## CategoryId: Anonimleştirilmiş kategorilerdir. (Örnek : Temizlik, nakliyat, tadilat kategorisi)
## CreateDate: Hizmetin satın alındığı tarih


In [2]:
# importing the libraries
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
import datetime as dt
warnings.filterwarnings('ignore')
from mlxtend.frequent_patterns import apriori, association_rules
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)
pd.set_option('display.float_format', lambda x: '%.2f' % x)
pd.set_option('display.expand_frame_repr', False)   # ensures that the output is on a single line

In [3]:
armut=pd.read_csv("/kaggle/input/dataset1/armut_data.csv")
armut.head()

,UserId,ServiceId,CategoryId,CreateDate
0,25446,4,5,2017-08-06 16:11:00
1,22948,48,5,2017-08-06 16:12:00
2,10618,0,8,2017-08-06 16:13:00
3,7256,9,4,2017-08-06 16:14:00
4,25446,48,5,2017-08-06 16:16:00


In [4]:
df=armut.copy()

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 162523 entries, 0 to 162522
Data columns (total 4 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   UserId      162523 non-null  int64 
 1   ServiceId   162523 non-null  int64 
 2   CategoryId  162523 non-null  int64 
 3   CreateDate  162523 non-null  object
dtypes: int64(3), object(1)
memory usage: 5.0+ MB


In [6]:
change=[i for i in df.columns if "Id" in i][1::]
df[change]=df[change].astype(str)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 162523 entries, 0 to 162522
Data columns (total 4 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   UserId      162523 non-null  int64 
 1   ServiceId   162523 non-null  object
 2   CategoryId  162523 non-null  object
 3   CreateDate  162523 non-null  object
dtypes: int64(1), object(3)
memory usage: 5.0+ MB


In [7]:
df["Hizmet"]= [row[1]+ "_" + row[2] for row in df.values]

In [8]:
df

,UserId,ServiceId,CategoryId,CreateDate,Hizmet
0,25446,4,5,2017-08-06 16:11:00,4_5
1,22948,48,5,2017-08-06 16:12:00,48_5
2,10618,0,8,2017-08-06 16:13:00,0_8
3,7256,9,4,2017-08-06 16:14:00,9_4
4,25446,48,5,2017-08-06 16:16:00,48_5
...,...,...,...,...,...
162518,10591,25,0,2018-08-06 14:40:00,25_0
162519,10591,2,0,2018-08-06 14:43:00,2_0
162520,10591,31,6,2018-08-06 14:47:00,31_6
162521,12666,38,4,2018-08-06 16:01:00,38_4


In [9]:
df['CreateDate'].dtypes

dtype('O')

In [10]:
df['CreateDate'].apply(lambda x:x[0:7])

0         2017-08
1         2017-08
2         2017-08
3         2017-08
4         2017-08
           ...   
162518    2018-08
162519    2018-08
162520    2018-08
162521    2018-08
162522    2018-08
Name: CreateDate, Length: 162523, dtype: object

In [11]:
new=pd.to_datetime(df['CreateDate']).dt.strftime("%Y-%m")
new

0         2017-08
1         2017-08
2         2017-08
3         2017-08
4         2017-08
           ...   
162518    2018-08
162519    2018-08
162520    2018-08
162521    2018-08
162522    2018-08
Name: CreateDate, Length: 162523, dtype: object

In [12]:
df["New_Date"]=new

In [13]:
df

,UserId,ServiceId,CategoryId,CreateDate,Hizmet,New_Date
0,25446,4,5,2017-08-06 16:11:00,4_5,2017-08
1,22948,48,5,2017-08-06 16:12:00,48_5,2017-08
2,10618,0,8,2017-08-06 16:13:00,0_8,2017-08
3,7256,9,4,2017-08-06 16:14:00,9_4,2017-08
4,25446,48,5,2017-08-06 16:16:00,48_5,2017-08
...,...,...,...,...,...,...
162518,10591,25,0,2018-08-06 14:40:00,25_0,2018-08
162519,10591,2,0,2018-08-06 14:43:00,2_0,2018-08
162520,10591,31,6,2018-08-06 14:47:00,31_6,2018-08
162521,12666,38,4,2018-08-06 16:01:00,38_4,2018-08


In [14]:
df["Sepet_Id"]= [str(row[0])+ "_" + row[-1] for row in df.values]

In [15]:
df

,UserId,ServiceId,CategoryId,CreateDate,Hizmet,New_Date,Sepet_Id
0,25446,4,5,2017-08-06 16:11:00,4_5,2017-08,25446_2017-08
1,22948,48,5,2017-08-06 16:12:00,48_5,2017-08,22948_2017-08
2,10618,0,8,2017-08-06 16:13:00,0_8,2017-08,10618_2017-08
3,7256,9,4,2017-08-06 16:14:00,9_4,2017-08,7256_2017-08
4,25446,48,5,2017-08-06 16:16:00,48_5,2017-08,25446_2017-08
...,...,...,...,...,...,...,...
162518,10591,25,0,2018-08-06 14:40:00,25_0,2018-08,10591_2018-08
162519,10591,2,0,2018-08-06 14:43:00,2_0,2018-08,10591_2018-08
162520,10591,31,6,2018-08-06 14:47:00,31_6,2018-08,10591_2018-08
162521,12666,38,4,2018-08-06 16:01:00,38_4,2018-08,12666_2018-08


# Adım 3: Veri seti hizmetlerin alındığı tarih ve saatten oluşmaktadır, herhangi bir sepet tanımı (fatura vb. ) bulunmamaktadır.
# Association Rule Learning uygulayabilmek için bir sepet (fatura vb.) tanımı oluşturulması gerekmektedir.
# Burada sepet tanımı her bir müşterinin aylık aldığı hizmetlerdir. Örneğin; 7256 id'li müşteri 2017'in 8.ayında aldığı 9_4, 46_4 hizmetleri bir sepeti;
# 2017’in 10.ayında aldığı  9_4, 38_4  hizmetleri başka bir sepeti ifade etmektedir. Sepetleri unique bir ID ile tanımlanması gerekmektedir.
# Bunun için öncelikle sadece yıl ve ay içeren yeni bir date değişkeni oluşturunuz. UserID ve yeni oluşturduğunuz date değişkenini "_"
# ile birleştirirek ID adında yeni bir değişkene atayınız.

In [16]:
df.groupby(['Sepet_Id', 'Hizmet'])["Hizmet"].count().unstack().fillna(0).iloc[0:5,0:5]


Hizmet,0_8,10_9,11_11,12_7,13_11
Sepet_Id,,,,,
0_2017-08,0.00,0.00,0.00,0.00,0.00
0_2017-09,0.00,0.00,0.00,0.00,0.00
0_2018-01,0.00,0.00,0.00,0.00,0.00
0_2018-04,0.00,0.00,0.00,0.00,0.00
10000_2017-08,0.00,0.00,0.00,0.00,0.00


In [21]:
invoices=df.groupby(['Sepet_Id', 'Hizmet'])["Hizmet"].count().unstack().fillna(0).applymap(lambda x: 1 if x > 0 else 0)
invoices.head()

Hizmet,0_8,10_9,11_11,12_7,13_11,14_7,15_1,16_8,17_5,18_4,19_6,1_4,20_5,21_5,22_0,23_10,24_10,25_0,26_7,27_7,28_4,29_0,2_0,30_2,31_6,32_4,33_4,34_6,35_11,36_1,37_0,38_4,39_10,3_5,40_8,41_3,42_1,43_2,44_0,45_6,46_4,47_7,48_5,49_1,4_5,5_11,6_7,7_3,8_5,9_4
Sepet_Id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0_2017-08,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0
0_2017-09,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0
0_2018-01,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0
0_2018-04,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
10000_2017-08,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0


## Association Rules

* After creating the pivot table, now we can make the association rule analysis using apriori and association_rules methods.
* Finding the support values, i.e. probabilities, of all possible visual associations with the apriori method

In [22]:

from mlxtend.frequent_patterns import apriori, association_rules
frequent_itemsets = apriori(invoices, min_support=0.01, use_colnames=True)


/opt/conda/lib/python3.10/site-packages/mlxtend/frequent_patterns/fpcommon.py:110: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(


In [23]:
frequent_itemsets.head()

,support,itemsets
0,0.02,(0_8)
1,0.03,(11_11)
2,0.03,(12_7)
3,0.06,(13_11)
4,0.02,(14_7)


In [26]:
rules = association_rules(frequent_itemsets, metric='support', min_threshold=0.01)

In [31]:
rules.sort_values(by='lift')

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
9,(49_1),(15_1),0.07,0.12,0.01,0.15,1.22,0.00,1.03,0.19
8,(15_1),(49_1),0.12,0.07,0.01,0.08,1.22,0.00,1.02,0.21
17,(38_4),(2_0),0.07,0.13,0.01,0.17,1.29,0.00,1.05,0.24
16,(2_0),(38_4),0.13,0.07,0.01,0.09,1.29,0.00,1.02,0.26
7,(38_4),(15_1),0.07,0.12,0.01,0.17,1.39,0.00,1.06,0.30
6,(15_1),(38_4),0.12,0.07,0.01,0.09,1.39,0.00,1.03,0.32
0,(2_0),(13_11),0.13,0.06,0.01,0.10,1.74,0.01,1.05,0.49
1,(13_11),(2_0),0.06,0.13,0.01,0.23,1.74,0.01,1.12,0.45
3,(2_0),(15_1),0.13,0.12,0.03,0.26,2.15,0.02,1.19,0.62
2,(15_1),(2_0),0.12,0.13,0.03,0.28,2.15,0.02,1.21,0.61


In [40]:
hizmet=df[df["Hizmet"]=='2_0']

In [44]:
def arl_recommender(rules, product_id, recommend_count=10):
    sorted_rules = rules.sort_values('lift', ascending=False)
    recommendation_list = []
    for i, product in enumerate(sorted_rules['antecedents']):
        for j in list(product):
            if j == product_id:
                recommendation_list.append(list(sorted_rules.iloc[i]['consequents'])[0])
    return recommendation_list[0: recommend_count]


arl_recommender(rules,'2_0')

['22_0', '25_0', '15_1', '13_11', '38_4']